<a href="https://colab.research.google.com/github/7caiorodrigues/Projects/blob/main/Analise_NBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Tratamentos iniciais

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Importando o dataset
df = pd.read_csv("all_seasons.csv")
df['draft_round'].unique()

In [ ]:
#Verificando as variáveis e o tipo de cada uma delas
df.info()

In [ ]:
df

In [ ]:
#Dropando a primeira coluna que seria tipo um indice
df = df.drop("Unnamed: 0", axis=1)

In [ ]:
df.head()

In [ ]:
df.pts.describe()

In [ ]:
#jogador mais velho da liga
df.query('age == 44')

In [ ]:
#Pegando a lista de nomes de times abreviados
abrev_times  = df.team_abbreviation.unique()
abrev_times

In [ ]:
#Contagem de jogadores por país

df_novo = df.country.value_counts()
df_novo

In [ ]:
#Media de pontos dos jogadores por país

df.groupby('country')['pts'].median()

In [ ]:
#Listando os jogadores brasileiros que atuaram na temporada 2021-22

df.query("country == 'Brazil' and season == '2021-22'")

In [ ]:
#Pegando os valores unicos de identificação de cada temporada
df.season.unique()

In [ ]:
df.info()

Analises:

- Pegar o maior pontuador de cada temporada;
- Ver em qual temporada houve a maior média de pontos.
- Qual a média de pontos dos seed #1 do draft em cada temporada
- Teve algum não draftado que fez mais de 20ppg em alguma temporada?
- Media de altura dos jogadores por temporada
- Media de pontos de cada round do draft por temporada
- Media de pontos por College na NBA
- Media de Idade dos maiores pontuadores de cada temporada

#### Maior pontuador de cada temporada

In [ ]:
#Criando um DataFrame que pega o valor maximo da coluna 'pts' agrupando pela coluna 'season'
df_pts = df.groupby('season')['pts'].max()
df_fato = pd.DataFrame(df_pts)
df_fato.info()

In [ ]:
df

In [ ]:
#Aqui eu pego o indice de cada valor considerado máximo para cada season e armazeno na variavel 'ow_index'
#A variavel 'related_name' vai pegar no DataFrame qual o valor da coluna 'player_name' está relacionado a este indice
#Depois crio um DataFrame com o nome dos jogadores

row_index = df.groupby('season')['pts'].idxmax()
name = df.loc[row_index, 'player_name']
nomes = pd.DataFrame(name)
nomes.info()

In [ ]:
#Aqui veio o minidesafio dessa parte, pois, ao simplesmente copiar a coluna de um DF para o outros, os valores apareciam
#no DF destino como NaN. Isso se deve pois os indices estavam trocados. Como sei que os DFs sao de tamanhos iguais,
#igualei os indices para que o problema se resolvesse.

nomes.index = df_fato.index
df_fato['Nomes'] = nomes['player_name']
df_fato

In [ ]:
#Print indicando cada saída
for index, row in df_fato.iterrows():
    print(f"O maior pontuador da temporada {index} foi o {row['Nomes']} com {row['pts']} ppg.")

#### Ver em qual temporada houve a maior média de pontos

In [ ]:
#Este comando pega a media de pontos agrupando pela variavel 'season' e depois o transforma num DataFrame

df_media = df.groupby('season')['pts'].mean()
df_media_per_season = pd.DataFrame(df_media)
df_media_per_season

In [ ]:
#Pega o maior valor entre as medias e em qual indice ele está.
max_value = df_media_per_season['pts'].max()
row_index = df_media_per_season['pts'].idxmax()


In [ ]:
print(f"A temporada com maior média de pontos na NBA foi a {row_index} com {max_value} pontos.")

####Qual a média de pontos dos seed 1 do draft em cada temporada que foi draftado

In [ ]:
#Verificando se havia alguem cadastrado com draft = 0

sel = df['draft_number'] == 0
sel.value_counts()

In [ ]:
#Verificando os dados unicos de numeros de draft e verificamos que os jogadores que nao foram draftados são colocados
#como 'UNdrafted'
df['draft_number'].unique()

In [ ]:
#Como verificamos acima, podemos substituir a palavra 'Undrafted' para 0, que ai poderemos transformar a variavel em
#int e trabalhar com ela abaixo.

df['draft_number'] = df['draft_number'].replace("Undrafted", 555555)
df['draft_year'] = df['draft_year'].replace("Undrafted", 555555)
df['draft_round'] = df['draft_round'].replace("Undrafted", 555555)

In [ ]:
#conferindo os valores unicos novamente pra ver se a substituição deu certo
df['draft_number'].unique()

In [ ]:
#Alterando o tipo da variavel da coluna pra INT
df['draft_number'] = df['draft_number'].astype(int)
print(df.dtypes)

In [ ]:
#Copiamos todos as linhas cuja coluna de 'draft_number' seja igual a 1
seed = 1
df_draft = df.loc[df['draft_number'] == seed].copy()

#Resetamos os indices para o novo DataFrame
df_draft.reset_index(drop=True, inplace=True)
df_draft

In [ ]:
#Pegamos os 4 primeiros digitos da coluna 'season' e geramos uma nova coluna que será util mais abaixo.
df_draft['compara_season_draft'] = df_draft['season'].astype(str).str[:4]
df_draft

In [ ]:
#geramos essa nova coluna no DataFrame geral também
df['compara_season_draft'] = df['season'].astype(str).str[:4]

In [ ]:
#alteramos o tipo da variavel para essa nova coluna
df['compara_season_draft'] = df['compara_season_draft'].astype(int)

In [ ]:
#Aqui pegamos todas as linhas que onde o 'draft_year' é igual ao 'compara_season_draft'
#Por isso criamos essa variavel, pra gente conseguir pegar somente o jogador n1 do draft daquele ano

df_draft_player_year = df_draft.loc[df_draft['draft_year'] == df_draft['compara_season_draft']]
df_draft_player_year

In [ ]:
#Aqui listamos apenas algumas features do novo DataFrame que criamos, para facilitar a visualização
list = ['player_name', 'draft_number', 'pts', 'season']

#resetamos os indices e ordenamos pela coluna de pts, de forma descendente.
df_draft_player_year = df_draft_player_year.reset_index(drop=True)
df_draft_player_year[list].sort_values(by='pts', ascending=False)

####Teve algum não draftado que fez mais de 20ppg em alguma temporada?

In [ ]:
df_undrafted = df.loc[df['draft_year'] == 555555]
df_undrafted

In [ ]:
df_undrafted_1 = df_undrafted.loc[df['pts'] > 15]
list_und = ['player_name', 'pts', 'draft_year', 'season']
df_undrafted_1[list_und]


####Media de altura dos jogadores por temporada

In [ ]:
df.info()

In [ ]:
df.groupby('season')['player_height'].mean()

####Media de pontos de cada round do draft por temporada

In [ ]:
rounds = df['draft_round'].unique()
rounds

In [ ]:
df.groupby("draft_round")['pts'].mean()

####Media de pontos por College na NBA



In [336]:
df_college = df.groupby('college')['pts'].mean()

In [ ]:
df_college = pd.DataFrame(df_college)
df_college.sort_values(by='pts', ascending=False)

In [ ]:
df_players_davidson = df.loc[df['college'] == "Davidson"]
df_players_davidson

###Media de idade geral dos maiores pontuadores de cada temporada

In [ ]:
#Agrupando os maiores pontuadores de cada temporada
df_best_scorers = df.groupby("season")['pts'].max()
df_best_scorers

In [354]:
#Pegando o indice de cada "melhor pontuador" e relacionando com a sua idade

row_index = df.groupby('season')['pts'].idxmax()
name = df.loc[row_index, 'age']
idade = pd.DataFrame(name)
idade

,age
392,34.0
479,35.0
1019,24.0
1554,28.0
1920,26.0
2489,27.0
2953,24.0
3181,25.0
3664,30.0
4342,27.0


In [355]:
#Unindo os dois dataframes
idade.index = df_best_scorers.index
df_best_scorers['idade'] = idade['age']
df_best_scorers

season
1996-97                                                 29.6
1997-98                                                 28.7
1998-99                                                 26.8
1999-00                                                 29.7
2000-01                                                 31.1
2001-02                                                 31.4
2002-03                                                 32.1
2003-04                                                 28.0
2004-05                                                 30.7
2005-06                                                 35.4
2006-07                                                 31.6
2007-08                                                 30.0
2008-09                                                 30.2
2009-10                                                 30.1
2010-11                                                 27.7
2011-12                                                 28.0
2012-13          

In [358]:
#Calculo a media de idade entre os maiores pontuadores.
media_best_scorers = df_best_scorers['idade'].mean()

In [359]:
print(f"A média geral da idade dos melhores pontuadores de cada temporada é: {round(media_best_scorers, 2)} anos.")

A média geral da idade dos melhores pontuadores de cada temporada é: 27.23 anos.
